In [1]:
import folium
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import requests
from sklearn.cluster import KMeans

In [2]:
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

In [3]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

tables = pd.read_html(link)

data = tables[0]
filter = data[data['Borough'] != 'Not assigned']
neighborhoods = filter.reset_index(drop=True)
geo_data = pd.read_csv('Geospatial_Coordinates.csv')

toronto_data = pd.merge(neighborhoods, geo_data, on='Postal Code')
toronto_neighborhoods = toronto_data.drop(columns=['Postal Code'])

In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
newyork_neighborhoods = pd.DataFrame(columns=column_names)

In [5]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    newyork_neighborhoods = newyork_neighborhoods.append({'Borough': borough,
                                                          'Neighborhood': neighborhood_name,
                                                          'Latitude': neighborhood_lat,
                                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
with open('quartier_paris.json') as json_data:
    paris_quartiers = json.load(json_data)
column_names = ['Arrondissement', 'Neighborhood', 'Latitude', 'Longitude']
paris_neighborhoods = pd.DataFrame(columns=column_names)
for i in paris_quartiers:
    arrondissement = i['fields']['c_ar']
    neighborhood_name = i['fields']['l_qu']
    latlon = i['fields']['geom_x_y']
    lat = latlon[0]
    lon = latlon[1]

    paris_neighborhoods = paris_neighborhoods.append({'Arrondissement': arrondissement,
                                                     'Neighborhood': neighborhood_name,
                                                     'Latitude': lat,
                                                     'Longitude': lon}, ignore_index=True)

In [7]:
address = ['Toronto, Canada', 'New York City, USA', 'Paris, France']
lat_lst = []
lon_lst = []

for i in address:
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(i)
    lat_lst.append(location.latitude)
    lon_lst.append(location.longitude)

hide_toggle(for_next=True)

In [8]:
CLIENT_ID = 'QJWNQ5SA1BXS2BQOR5GZKVG04BKENQ4YUBU3HR3BUMO2I0F2'
CLIENT_SECRET = 'PWQJ4Z1BHOWI3XH0XUQKRPKQ1MUDAGSXYBOLN1JTB3Y0WLJO'
VERSION = '20180605'

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=800, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
toronto_venues = getNearbyVenues(names=toronto_neighborhoods['Neighborhood'],
                                 latitudes=toronto_neighborhoods['Latitude'],
                                 longitudes=toronto_neighborhoods['Longitude'])

In [11]:
newyork_venues = getNearbyVenues(names=newyork_neighborhoods['Neighborhood'],
                                 latitudes=newyork_neighborhoods['Latitude'],
                                 longitudes=newyork_neighborhoods['Longitude'])

In [12]:
paris_venues = getNearbyVenues(names=paris_neighborhoods['Neighborhood'],
                               latitudes=paris_neighborhoods['Latitude'],
                               longitudes=paris_neighborhoods['Longitude'])

In [13]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [14]:
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]

In [15]:
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

paris_onehot['Neighborhood'] = paris_venues['Neighborhood'] 

fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

In [16]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()
paris_grouped = paris_onehot.groupby('Neighborhood').mean().reset_index()

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
venues = [toronto_grouped, newyork_grouped, paris_grouped]
sort = []
for i in venues:
    num_top_venues = 10
    
    indicators = ['st', 'nd', 'rd']
    columns = ['Neighborhood']
    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))

    neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
    neighborhoods_venues_sorted['Neighborhood'] = i['Neighborhood']

    for ind in np.arange(i.shape[0]):
        neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(i.iloc[ind, :],
                                                                              num_top_venues)
    kclusters = 5

    grouped_clustering = i.drop('Neighborhood', 1)

    kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)
    
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
    
    sort.append(neighborhoods_venues_sorted)

In [19]:
neighborhoods = [toronto_neighborhoods, newyork_neighborhoods, paris_neighborhoods]
merged_lst = []
maps = []
for i in range(3):
    merged = neighborhoods[i]
    
    merged = merged.join(sort[i].set_index('Neighborhood'), on='Neighborhood')
    
    merged = merged.dropna(axis=0)
    
    merged_lst.append(merged)
    
    map_clusters = folium.Map(location=[lat_lst[i], lon_lst[i]], zoom_start=11)

    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    markers_colors = []
    for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'],
                                      merged['Neighborhood'], merged['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster-1)],
            fill=True,
            fill_color=rainbow[int(cluster-1)],
            fill_opacity=0.7).add_to(map_clusters)
    maps.append(map_clusters)

In [20]:
maps[0]

In [21]:
merged_lst[0].loc[merged_lst[0]['Cluster Labels'] == 0, merged_lst[0].columns[[1] + list(range(5, merged_lst[0].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Bus Stop,Road,Park,Food & Drink Shop,Women's Store,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner
5,"Islington Avenue, Humber Valley Village",Pharmacy,Park,Playground,Café,Grocery Store,Shopping Mall,Bank,Skating Rink,Distribution Center,Design Studio
21,Caledonia-Fairbanks,Park,Women's Store,Japanese Restaurant,Pharmacy,Cosmetics Shop,Café,Mexican Restaurant,Beer Store,Bank,Bakery
22,Woburn,Coffee Shop,Park,Business Service,Dive Bar,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
39,Bayview Village,Japanese Restaurant,Bank,Café,Grocery Store,Dog Run,Skating Rink,Chinese Restaurant,Park,Shopping Mall,Design Studio
49,"North Park, Maple Leaf Park, Upwood Park",Tennis Court,Park,Trail,Bakery,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
66,York Mills West,Park,Restaurant,Convenience Store,Deli / Bodega,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
83,"Moore Park, Summerhill East",Park,Grocery Store,Japanese Restaurant,Thai Restaurant,Gym,Café,Candy Store,Sandwich Place,Diner,Deli / Bodega
91,Rosedale,Trail,Park,Playground,Grocery Store,Candy Store,Bank,Dumpling Restaurant,Elementary School,Design Studio,Dessert Shop
101,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Playground,Gym / Fitness Center,Baseball Field,Park,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


In [22]:
merged_lst[0].loc[merged_lst[0]['Cluster Labels'] == 1, merged_lst[0].columns[[1] + list(range(5, merged_lst[0].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,"York Mills, Silver Hills",Pool,Deli / Bodega,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run


In [23]:
merged_lst[0].loc[merged_lst[0]['Cluster Labels'] == 2, merged_lst[0].columns[[1] + list(range(5, merged_lst[0].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Glencairn,Pizza Place,Grocery Store,Fast Food Restaurant,Gas Station,Bus Line,Japanese Restaurant,Latin American Restaurant,Mediterranean Restaurant,Metro Station,Restaurant
11,"West Deane Park, Princess Gardens, Martin Grov...",Hotel,Convenience Store,Pizza Place,Café,Restaurant,Bank,Theater,Mexican Restaurant,Gym,Eastern European Restaurant
14,Woodbine Heights,Pizza Place,Video Store,Sandwich Place,Curling Ice,Bus Line,Park,Skating Rink,Athletics & Sports,Beer Store,Pharmacy
18,"Guildwood, Morningside, West Hill",Pizza Place,Restaurant,Fast Food Restaurant,Beer Store,Fried Chicken Joint,Salon / Barbershop,Sports Bar,Supermarket,Bank,Coffee Shop
32,Scarborough Village,Ice Cream Shop,Coffee Shop,Fast Food Restaurant,Convenience Store,Restaurant,Sandwich Place,Pizza Place,Eastern European Restaurant,Discount Store,Deli / Bodega
51,"Cliffside, Cliffcrest, Scarborough Village West",Pizza Place,Ice Cream Shop,Hardware Store,Restaurant,Park,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner
56,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Discount Store,Bar,Grocery Store,Sandwich Place,Dessert Shop,Gas Station,Convenience Store,Auto Garage,Wine Shop,Dumpling Restaurant
57,"Humberlea, Emery",Convenience Store,Baseball Field,Gas Station,Shipping Store,Shoe Store,Discount Store,Park,Furniture / Home Store,Dog Run,Doner Restaurant
63,"Runnymede, The Junction North",Brewery,Pizza Place,Coffee Shop,Park,Athletics & Sports,Gas Station,Furniture / Home Store,Bus Line,Sandwich Place,Discount Store
64,Weston,Convenience Store,Breakfast Spot,Fried Chicken Joint,Pizza Place,Diner,Pharmacy,Bank,Park,Dim Sum Restaurant,Discount Store


In [24]:
merged_lst[0].loc[merged_lst[0]['Cluster Labels'] == 3, merged_lst[0].columns[[1] + list(range(5, merged_lst[0].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,"Northwest, West Humber - Clairville",Hotel,Rental Car Location,Doner Restaurant,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar


In [25]:
merged_lst[0].loc[merged_lst[0]['Cluster Labels'] == 4, merged_lst[0].columns[[1] + list(range(5, merged_lst[0].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Coffee Shop,Hockey Arena,Pizza Place,Portuguese Restaurant,Park,Café,Sporting Goods Shop,Grocery Store,Diner,Curling Ice
2,"Regent Park, Harbourfront",Coffee Shop,Theater,Park,Bakery,Restaurant,Café,Italian Restaurant,Pub,Breakfast Spot,Thai Restaurant
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Furniture / Home Store,Fried Chicken Joint,Accessories Store,Dessert Shop,Vietnamese Restaurant,Fast Food Restaurant,Coffee Shop,Bowling Alley,Athletics & Sports
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Café,Park,Bookstore,Bubble Tea Shop,Gastropub,Italian Restaurant,Japanese Restaurant,Diner,Creperie
6,"Malvern, Rouge",Trail,Fast Food Restaurant,Chinese Restaurant,Hobby Shop,Auto Workshop,Paper / Office Supplies Store,African Restaurant,Bus Station,Coffee Shop,Spa
...,...,...,...,...,...,...,...,...,...,...,...
97,"First Canadian Place, Underground city",Hotel,Coffee Shop,Café,Restaurant,Japanese Restaurant,Theater,Concert Hall,Vegetarian / Vegan Restaurant,Italian Restaurant,Seafood Restaurant
98,"The Kingsway, Montgomery Road, Old Mill North",Italian Restaurant,Bank,Sushi Restaurant,Breakfast Spot,Burger Joint,Dessert Shop,Bakery,French Restaurant,Bar,River
99,Church and Wellesley,Coffee Shop,Japanese Restaurant,Men's Store,Park,Ramen Restaurant,Café,Diner,Sushi Restaurant,Dance Studio,Thai Restaurant
100,"Business reply mail Processing Centre, South C...",Fast Food Restaurant,Light Rail Station,Bakery,Harbor / Marina,Brewery,Burrito Place,Restaurant,Coffee Shop,Intersection,Italian Restaurant


In [26]:
maps[1]

In [27]:
merged_lst[1].loc[merged_lst[1]['Cluster Labels'] == 0, merged_lst[1].columns[[1] + list(range(5, merged_lst[1].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Clason Point,Park,Bus Stop,Trail,River,Boat or Ferry,Pool,Convenience Store,Fast Food Restaurant,Film Studio,Filipino Restaurant
203,Todt Hill,Park,Mountain,Dog Run,Yoga Studio,Fish & Chips Shop,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market
294,Malba,Park,Rest Area,Scenic Lookout,Bus Station,Chinese Restaurant,Harbor / Marina,Pizza Place,Playground,Vegetarian / Vegan Restaurant,Food Truck
303,Bayswater,Park,Auto Dealership,Playground,Home Service,Yoga Studio,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm


In [28]:
merged_lst[1].loc[merged_lst[1]['Cluster Labels'] == 1, merged_lst[1].columns[[1] + list(range(5, merged_lst[1].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Hunts Point,Waste Facility,Bank,Spanish Restaurant,Paper / Office Supplies Store,Gourmet Shop,Park,Market,Pool,Construction & Landscaping,BBQ Joint
42,Pelham Gardens,Bus Station,Deli / Bodega,Chinese Restaurant,Pharmacy,Donut Shop,Spa,Spanish Restaurant,Mobile Phone Shop,Plaza,Bus Line
72,East New York,Deli / Bodega,Video Game Store,Bus Station,Fried Chicken Joint,Pet Store,Caribbean Restaurant,Playground,Plaza,Auto Garage,Hotel
89,Ocean Hill,Discount Store,Fried Chicken Joint,Café,Deli / Bodega,Fast Food Restaurant,Health & Beauty Service,Playground,Convenience Store,Chinese Restaurant,Bus Stop
148,South Ozone Park,Park,Deli / Bodega,Chinese Restaurant,Donut Shop,Hotel,Food,Breakfast Spot,Moving Target,Fast Food Restaurant,Bar
154,Douglaston,Chinese Restaurant,Deli / Bodega,Italian Restaurant,Bank,Spa,Japanese Restaurant,Greek Restaurant,Pharmacy,Sandwich Place,Coffee Shop
161,Oakland Gardens,Chinese Restaurant,Pharmacy,Korean Restaurant,Yoga Studio,Sushi Restaurant,Deli / Bodega,Donut Shop,Bank,Italian Restaurant,Liquor Store
163,Hollis,Deli / Bodega,Fried Chicken Joint,Pizza Place,Fast Food Restaurant,Bus Stop,Supermarket,Shopping Mall,Lounge,Liquor Store,Sandwich Place
165,St. Albans,Caribbean Restaurant,Deli / Bodega,Chinese Restaurant,Fried Chicken Joint,Video Store,Pizza Place,Train,Fast Food Restaurant,Liquor Store,Fish & Chips Shop
166,Rochdale,Pharmacy,Deli / Bodega,Bus Stop,Southern / Soul Food Restaurant,Fast Food Restaurant,Market,Japanese Restaurant,Sandwich Place,Breakfast Spot,Food


In [29]:
merged_lst[1].loc[merged_lst[1]['Cluster Labels'] == 2, merged_lst[1].columns[[1] + list(range(5, merged_lst[1].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Pharmacy,Gas Station,Supermarket,Pizza Place,Donut Shop,Caribbean Restaurant,Ice Cream Shop,Gift Shop,Bagel Shop,Sandwich Place
1,Co-op City,Mobile Phone Shop,Supermarket,Shoe Store,Pizza Place,Fast Food Restaurant,Discount Store,Mattress Store,Bus Station,Accessories Store,Donut Shop
2,Eastchester,Caribbean Restaurant,Fast Food Restaurant,Pharmacy,Donut Shop,Diner,Grocery Store,Deli / Bodega,Bakery,Seafood Restaurant,Cocktail Bar
5,Kingsbridge,Pizza Place,Bar,Mexican Restaurant,Diner,Donut Shop,Bakery,Supermarket,Sandwich Place,Coffee Shop,Deli / Bodega
6,Marble Hill,Pizza Place,Spanish Restaurant,Sandwich Place,Donut Shop,Bus Station,Café,Gym,Supermarket,Coffee Shop,Park
...,...,...,...,...,...,...,...,...,...,...,...
297,Bronxdale,Pizza Place,Italian Restaurant,Bank,Supermarket,Coffee Shop,Sandwich Place,Chinese Restaurant,Deli / Bodega,Donut Shop,Pharmacy
298,Allerton,Pizza Place,Donut Shop,Pharmacy,Sandwich Place,Supermarket,Caribbean Restaurant,Fast Food Restaurant,Bar,Fried Chicken Joint,Discount Store
299,Kingsbridge Heights,Coffee Shop,Pizza Place,Park,Chinese Restaurant,Latin American Restaurant,Bus Station,Grocery Store,Food Truck,Lake,Deli / Bodega
300,Erasmus,Caribbean Restaurant,Juice Bar,Liquor Store,Pizza Place,Lounge,Yoga Studio,Furniture / Home Store,Bakery,Bank,Gym / Fitness Center


In [30]:
merged_lst[1].loc[merged_lst[1]['Cluster Labels'] == 3, merged_lst[1].columns[[1] + list(range(5, merged_lst[1].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Bus Station,Park,Athletics & Sports,Plaza,Bar,Art Gallery,Coffee Shop,Bakery,Historic Site,Pizza Place
4,Riverdale,Bar,Diner,Plaza,Bank,Park,Mexican Restaurant,Pizza Place,Japanese Restaurant,Deli / Bodega,Bagel Shop
12,City Island,Harbor / Marina,Italian Restaurant,Boat or Ferry,Deli / Bodega,American Restaurant,Bar,Thrift / Vintage Store,Seafood Restaurant,Grocery Store,Park
39,Edgewater Park,Italian Restaurant,Deli / Bodega,Pizza Place,Spa,Latin American Restaurant,Bar,Asian Restaurant,Japanese Restaurant,Coffee Shop,Farmers Market
46,Bay Ridge,Pizza Place,Spa,Italian Restaurant,Bar,Bagel Shop,Cosmetics Shop,Chinese Restaurant,Greek Restaurant,Diner,Coffee Shop
...,...,...,...,...,...,...,...,...,...,...,...
286,Sandy Ground,Bar,Racetrack,Art Gallery,Bagel Shop,Intersection,Playground,Film Studio,Filipino Restaurant,Field,Fast Food Restaurant
288,Roxbury,Baseball Field,Theater,Sports Club,Irish Pub,Deli / Bodega,Fast Food Restaurant,National Park,Public Art,Beach,Scenic Lookout
292,Lighthouse Hill,Art Museum,Café,Bagel Shop,Trail,Golf Course,Spa,Clothing Store,Italian Restaurant,Food & Drink Shop,Food
301,Hudson Yards,Gym / Fitness Center,Hotel,Theater,Italian Restaurant,Gym,Art Gallery,Indie Theater,Coffee Shop,American Restaurant,Wine Shop


In [31]:
merged_lst[1].loc[merged_lst[1]['Cluster Labels'] == 4, merged_lst[1].columns[[1] + list(range(5, merged_lst[1].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,Sea Gate,Beach,Pizza Place,Park,Home Service,Gastropub,Spa,Supermarket,Farm,Falafel Restaurant,Film Studio
179,Neponsit,Beach,Pizza Place,Bus Stop,Park,Yoga Studio,Financial or Legal Service,Eye Doctor,Factory,Falafel Restaurant,Farm
190,Belle Harbor,Beach,Spa,Deli / Bodega,Pub,Boutique,Smoke Shop,Bakery,Bagel Shop,Chinese Restaurant,Pharmacy
204,South Beach,Beach,Pier,Deli / Bodega,American Restaurant,Soccer Field,Playground,Athletics & Sports,Park,Baseball Field,Skate Park
232,Midland Beach,Beach,Restaurant,Asian Restaurant,Playground,Pet Store,Chinese Restaurant,Other Great Outdoors,Business Service,Bus Stop,Bookstore


In [32]:
maps[2]

In [33]:
merged_lst[2].loc[merged_lst[2]['Cluster Labels'] == 0, merged_lst[2].columns[[1] + list(range(5, merged_lst[2].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arsenal,French Restaurant,Italian Restaurant,Hotel,Plaza,Cocktail Bar,Ice Cream Shop,Thai Restaurant,Pedestrian Plaza,Pizza Place,Asian Restaurant
1,Jardin-des-Plantes,French Restaurant,Café,Science Museum,Garden,Museum,Korean Restaurant,Hotel,Zoo Exhibit,Coffee Shop,Bakery
8,Saint-Germain-des-Prés,French Restaurant,Italian Restaurant,Hotel,Plaza,Café,Wine Bar,Japanese Restaurant,Seafood Restaurant,Cheese Shop,Bistro
16,Saint-Merri,French Restaurant,Bakery,Ice Cream Shop,Burger Joint,Coffee Shop,Plaza,Gay Bar,Cocktail Bar,Art Gallery,Gourmet Shop
17,Notre-Dame,French Restaurant,Ice Cream Shop,Plaza,Wine Bar,Park,Pastry Shop,Seafood Restaurant,Coffee Shop,Bookstore,Bakery
21,Halles,French Restaurant,Bakery,Cocktail Bar,Plaza,Chinese Restaurant,Historic Site,Pedestrian Plaza,Coffee Shop,Italian Restaurant,Theater
22,Sorbonne,French Restaurant,Plaza,Indie Movie Theater,Bakery,Bookstore,Wine Bar,Garden,Fountain,Mexican Restaurant,Café
30,Monnaie,French Restaurant,Plaza,Pub,Wine Bar,Café,Bookstore,Hotel,Ice Cream Shop,Cocktail Bar,Italian Restaurant
31,Odéon,French Restaurant,Plaza,Italian Restaurant,Bakery,Hotel,Ice Cream Shop,Indie Movie Theater,Chocolate Shop,Pub,Wine Bar
46,Saint-Victor,French Restaurant,Plaza,Museum,Coffee Shop,Italian Restaurant,Bakery,Bar,Ice Cream Shop,Comic Shop,Pub


In [34]:
merged_lst[2].loc[merged_lst[2]['Cluster Labels'] == 1, merged_lst[2].columns[[1] + list(range(5, merged_lst[2].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Porte-Saint-Martin,French Restaurant,Coffee Shop,Bistro,Restaurant,Cocktail Bar,Italian Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant,Pizza Place,Bar
6,Vivienne,Japanese Restaurant,French Restaurant,Hotel,Cocktail Bar,Bistro,Coffee Shop,Wine Bar,Bookstore,Theater,Korean Restaurant
7,Enfants-Rouges,Art Gallery,Restaurant,Bistro,Japanese Restaurant,French Restaurant,Wine Bar,Burger Joint,Sandwich Place,Italian Restaurant,Bookstore
9,Saint-Vincent-de-Paul,French Restaurant,Indian Restaurant,Hotel,Japanese Restaurant,Restaurant,Italian Restaurant,African Restaurant,Bistro,Coffee Shop,Café
11,Bel-Air,Bike Rental / Bike Share,Comedy Club,Bistro,Botanical Garden,Stadium,Sports Club,Café,Castle,Restaurant,Italian Restaurant
15,Quinze-Vingts,French Restaurant,Italian Restaurant,Bar,Hotel,Cocktail Bar,Sandwich Place,Nightclub,Bakery,Bookstore,Plaza
20,Bercy,Hotel,French Restaurant,Italian Restaurant,Garden,Supermarket,Bistro,Wine Bar,Japanese Restaurant,Plaza,Beer Garden
24,Chaussée-d'Antin,French Restaurant,Hotel,Gourmet Shop,Department Store,Italian Restaurant,Plaza,Clothing Store,Bistro,Bookstore,Café
25,Palais-Royal,Japanese Restaurant,French Restaurant,Italian Restaurant,Hotel,Plaza,Historic Site,Wine Bar,Café,Garden,Tea Room
27,Faubourg-Montmartre,French Restaurant,Hotel,Bakery,Wine Bar,Vegetarian / Vegan Restaurant,Bar,Japanese Restaurant,Bistro,Restaurant,Coffee Shop


In [35]:
merged_lst[2].loc[merged_lst[2]['Cluster Labels'] == 2, merged_lst[2].columns[[1] + list(range(5, merged_lst[2].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Plaisance,French Restaurant,Hotel,Bakery,Gym,Japanese Restaurant,Grocery Store,Pizza Place,Café,Bistro,Thai Restaurant
12,Montparnasse,French Restaurant,Hotel,Creperie,Italian Restaurant,Pizza Place,Ice Cream Shop,Wine Shop,Seafood Restaurant,Bar,Art Museum
13,Plaine de Monceaux,French Restaurant,Hotel,Bakery,Italian Restaurant,Plaza,Restaurant,Bistro,Japanese Restaurant,Pizza Place,Seafood Restaurant
14,Batignolles,French Restaurant,Italian Restaurant,Restaurant,Bar,Hotel,Turkish Restaurant,Pizza Place,Burger Joint,Coffee Shop,Park
18,Gros-Caillou,French Restaurant,Hotel,Italian Restaurant,Art Museum,Cocktail Bar,Coffee Shop,Plaza,Monument / Landmark,Tea Room,Café
19,Rochechouart,French Restaurant,Hotel,Bakery,Vegetarian / Vegan Restaurant,Coffee Shop,Pizza Place,Plaza,Italian Restaurant,Wine Bar,Bar
23,Saint-Georges,French Restaurant,Italian Restaurant,Hotel,Bakery,Wine Bar,Coffee Shop,Japanese Restaurant,Bar,Bistro,Seafood Restaurant
26,Invalides,French Restaurant,Hotel,Plaza,Art Museum,History Museum,Historic Site,Coffee Shop,Café,Bakery,Italian Restaurant
29,Amérique,French Restaurant,Supermarket,Hotel,Pizza Place,Grocery Store,Recording Studio,Food & Drink Shop,Beer Garden,Bistro,Market
34,Croulebarbe,French Restaurant,Hotel,Bakery,Thai Restaurant,Café,Italian Restaurant,Japanese Restaurant,Dessert Shop,Bar,Mexican Restaurant


In [36]:
merged_lst[2].loc[merged_lst[2]['Cluster Labels'] == 3, merged_lst[2].columns[[1] + list(range(5, merged_lst[2].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Roquette,French Restaurant,Bar,Italian Restaurant,Café,Bistro,Cocktail Bar,Wine Bar,Record Shop,Restaurant,Pizza Place
10,Saint-Ambroise,French Restaurant,Bar,Pizza Place,Restaurant,Coffee Shop,Bakery,Bistro,Wine Bar,Moroccan Restaurant,Ethiopian Restaurant
33,Maison-Blanche,Vietnamese Restaurant,Thai Restaurant,French Restaurant,Bakery,Asian Restaurant,Bar,Chinese Restaurant,Park,Café,Italian Restaurant
40,Père-Lachaise,Bar,French Restaurant,Café,Bakery,Japanese Restaurant,Bistro,Italian Restaurant,Theater,Restaurant,Plaza
44,Belleville,Bar,French Restaurant,Restaurant,Chinese Restaurant,Cocktail Bar,Pizza Place,Café,Italian Restaurant,Bistro,Vietnamese Restaurant
59,Clignancourt,French Restaurant,Bar,Bistro,Pizza Place,Café,Restaurant,Plaza,Italian Restaurant,Sandwich Place,Deli / Bodega
60,Goutte-d'Or,Bar,French Restaurant,Supermarket,Bistro,Convenience Store,Café,Bakery,Coffee Shop,Plaza,Pizza Place
62,Sainte-Marguerite,French Restaurant,Bar,Italian Restaurant,Bistro,Vietnamese Restaurant,Wine Bar,Thai Restaurant,Café,Hotel,Savoyard Restaurant
74,Folie-Méricourt,French Restaurant,Bar,Restaurant,Coffee Shop,Pizza Place,Bistro,Wine Bar,Vietnamese Restaurant,Cocktail Bar,Bakery
77,Combat,French Restaurant,Bar,Chinese Restaurant,Café,Restaurant,Asian Restaurant,Bistro,Italian Restaurant,Japanese Restaurant,Plaza


In [37]:
merged_lst[2].loc[merged_lst[2]['Cluster Labels'] == 4, merged_lst[2].columns[[1] + list(range(5, merged_lst[2].shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Picpus,Forest,Recreation Center,Bike Rental / Bike Share,National Park,Zoo,Food Truck,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
